# Homework 3

- Implement the ridge regression algorithm through a gradient descent algorithm
- apply it to a regression problem (with at least 10 predictors) for a chosen dataset
- optimize the choice of lambda with R^2 and cross-validation 
- check the accuracy of the final prediction on the test part of the data set.
- compare to a result of a simple linear regression fit
- comment on the results

In [20]:
import numpy as np
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import mean_absolute_error, classification_report,accuracy_score,r2_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize']=12,10

In [34]:
Xy = np.loadtxt("winequality-white.csv", delimiter=";", skiprows=1)
X = Xy[:, 0:-1]
X = scale(X)

y = Xy[:, -1]
y = scale(y)

np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
print(X_train)
print(X_train.shape)

[[ 4.09124986e-01 -1.80991745e-01 -1.17277959e-01 ...  2.76406991e-01
   1.34184656e-03 -1.15945603e-02]
 [-1.72412724e+00  1.16673788e-01  3.78559282e-01 ...  6.07565312e-01
  -1.13786799e+00  1.53249956e+00]
 [-4.20473102e-01  1.80344514e+00 -1.99917499e-01 ...  4.75101984e-01
   9.65288632e-01  8.01086557e-01]
 ...
 [ 1.23872307e+00  5.13561164e-01 -4.47836120e-01 ... -2.53446321e-01
  -9.62604939e-01 -1.39315246e+00]
 [ 2.42386320e+00 -1.80991745e-01 -3.46384190e-02 ... -3.85909649e-01
  -2.61552731e-01  2.32209775e-01]
 [ 1.23872307e+00  2.29955436e+00  4.80011213e-02 ... -6.50836305e-01
   1.76604898e-01 -4.17935119e-01]]
(3918, 11)


In [36]:
class RidgeRegression(object):
    def __init__(self, lmbda=0.1):
        self.lmbda = lmbda

    def fit(self, X, y):
        C = X.T.dot(X) + self.lmbda*np.eye(X.shape[1])
        self.w = np.linalg.inv(C).dot(X.T.dot(y))

    def predict(self, X):
        return X.dot(self.w)

    def get_params(self, deep=True):
        return {"lmbda": self.lmbda}

    def set_params(self, lmbda=0.1):
        self.lmbda = lmbda
        return self

In [106]:
class RidgeRegressionWithGrad(object):
    def __init__(self, lmbda=0.01):
        self.lmbda = lmbda
        self.lr = 0.001
        self.error_margin = 0.0001
        self.r2 = 1

    def fit(self, X, y):
        self.b0 = np.random.uniform(-1.0,1.0)
        self.b1= np.random.uniform(-1.0,1.0,X.shape[1])
        self.b1/=np.linalg.norm(self.b1)
        iterations = 0
        while True:
    
            rss0 = -2 * sum(y[i] - (self.b0+self.b1.T.dot(X[i])) for i in range(X.shape[0])) /X.shape[0]
            
            for j in range(len(self.b1)):
                rss1 = -2 * sum( X[i,j] * (y[i]- ( self.b0+ self.b1.dot(X[i] )) ) + 2*self.lmbda*self.b1[j] \
                                for i in range(X.shape[0])) /X.shape[0]
                self.b1[j] -= self.lr * rss1
            self.b1/=np.linalg.norm(self.b1)
            self.b0-=self.lr*rss0
            if iterations % 10 == 0:
                rr2 = self.score(X,y)

            if np.fabs(rr2-self.r2) < self.error_margin:
                break
            self.r2=rr2
            iterations+=1

    def predict(self, X):
        return np.repeat(self.b0,X.shape[0])+X.dot(self.b1)

    def get_params(self, deep=True):
        return {"lmbda": self.lmbda}

    def set_params(self, lmbda=0.01):
        self.lmbda = lmbda
        return self
    
    def score(self, X, y):
        mean = y.mean()
        TSS = sum ( (y[i] - mean)**2  for i in range(X.shape[0]) )
        RSS = sum ( ( y[i] - (self.b0+ self.b1.dot(X[i]) )  ) **2 for i in range(X.shape[0]) )
        return ( TSS - RSS ) / TSS

In [107]:
ridge = RidgeRegressionWithGrad(100)
ridge.fit(X_train, y_train)

In [108]:
predictions = ridge.predict(X_test)
ridge_error = r2_score(y_test, predictions)
print(ridge_error)

-2.066267638797712


In [90]:
ridge = RidgeRegression()
ridge.fit(X_train, y_train)
predictions = ridge.predict(X_test)
ridge_error = r2_score(y_test, predictions)
print(ridge_error)

0.25272478195695847


In [ ]:
#from sklearn.cross_validation import cross_val_score
ridge = RidgeRegression()
param_grid = [{"lmbda": 2.0**np.arange(-5, 10)}]
learner = GridSearchCV(ridge, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, verbose=0)
learner.fit(X_train, y_train)

predictions = learner.predict(X_test)
score = cross_val_score(learner, X, y, scoring='r2')
print(score.mean())

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
ridge_error = r2_score(y_test, predictions)
print(ridge_error)

In [ ]:
# Run for different values of lambda 
lambda_min = -7
lambda_max = 7

num_lambdas = 15
num_predictors = X.shape[1]

lambdas= np.linspace(lambda_min,lambda_max, num_lambdas)

train_r_squared = np.zeros(num_lambdas)
test_r_squared = np.zeros(num_lambdas)

coeff_a = np.zeros((num_lambdas, num_predictors))

for ind, i in enumerate(lambdas):    
    # Fit ridge regression on train set
    reg = RidgeRegression(10**i)
    reg.fit(X_train, y_train)
       
    coeff_a[ind,:] = reg.lmbda
    # Evaluate train & test performance
    train_r_squared[ind] = cross_val_score(learner, X_train, y_train, scoring='r2').mean()
    test_r_squared[ind] = cross_val_score(learner, X_test, y_test, scoring='r2').mean()
    
# Plotting
plt.figure(figsize=(18, 8))
plt.plot(lambdas, train_r_squared, 'bo-', label=r'$R^2$ Training set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(lambdas, test_r_squared, 'bo-', label=r'$R^2$ Test set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Lamda value'); plt.ylabel(r'$R^2$')
plt.xlim(lambda_min, lambda_max)
plt.title(r'Evaluate ridge regression $R^2$ with different lamdas')
plt.legend(loc='best')
plt.grid()